# Qwen聊天机器人
通义千问Qwen是阿里云自主研发的大语言模型，Qwen-1.5-0.5B 为其轻量版，拥有5亿参数，具备低成本和高效文本生成能力。适用于文字创作、文本处理、编程辅助、翻译服务和对话模拟等场景。

在本次任务中，我们将基于昇思MindSpore在香橙派开发板上运行通义千问Qwen-1.5-0.5B模型，体验和模型的对话互动，完成和聊天机器人对话。具体操作如下：

## 环境准备

开发者拿到香橙派开发板后，首先需要进行硬件资源确认，镜像烧录及CANN和MindSpore版本的升级，才可运行该案例，具体如下：

- 硬件： 香橙派AIpro 16G 8-12T开发板
- 镜像： 香橙派官网ubuntu镜像
- CANN：8.0.RC3.alpha002
- MindSpore： 2.4.10

### 镜像烧录

运行该案例需要烧录香橙派官网ubuntu镜像，烧录流程参考[昇思MindSpore官网--香橙派开发专区--环境搭建指南--镜像烧录](https://www.mindspore.cn/docs/zh-CN/r2.4.10/orange_pi/environment_setup.html#1-%E9%95%9C%E5%83%8F%E7%83%A7%E5%BD%95%E4%BB%A5windows%E7%B3%BB%E7%BB%9F%E4%B8%BA%E4%BE%8B)章节。

### CANN升级

CANN升级参考[昇思MindSpore官网--香橙派开发专区--环境搭建指南--CANN升级](https://www.mindspore.cn/docs/zh-CN/r2.4.10/orange_pi/environment_setup.html#3-cann%E5%8D%87%E7%BA%A7)章节。

### MindSpore升级

MindSpore升级参考[昇思MindSpore官网--香橙派开发专区--环境搭建指南--MindSpore升级](https://www.mindspore.cn/docs/zh-CN/r2.4.10/orange_pi/environment_setup.html#4-mindspore%E5%8D%87%E7%BA%A7)章节。

## 权重加载

此处使用mindnlp套件加载Qwen/Qwen1.5-0.5B-Chat模型权重，该套件包含了许多自然语言处理的常用方法，可以方便地加载和使用huggingface的模型权重。注意，首次运行时请耐心等待模型下载。

In [ ]:
#install mindnlp

!pip install mindnlp

In [1]:
import mindspore
from mindnlp.transformers import AutoModelForCausalLM, AutoTokenizer

# Loading the tokenizer and model from Hugging Face's model hub.
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen1.5-0.5B-Chat", ms_dtype=mindspore.float16)
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen1.5-0.5B-Chat", ms_dtype=mindspore.float16)

/home/HwHiAiUser/.conda/envs/mindspore_py39/lib/python3.9/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/HwHiAiUser/.conda/envs/mindspore_py39/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/HwHiAiUser/.conda/envs/mindspore_py39/lib/python3.9/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/HwHiAiUser/.conda/envs/mindspore_py39/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
Building prefix dict f

## 模型推理

模型加载完成后定义聊天机器人所需处理的事务：加载与处理聊天历史，转换成适合模型输入的格式；并通过流式生成器的推理模式逐步生成聊天机器人的回复。

In [2]:
from mindnlp.transformers import TextIteratorStreamer
from threading import Thread

system_prompt = "You are a helpful and friendly chatbot"

def build_input_from_chat_history(chat_history, msg: str):
    messages = [{'role': 'system', 'content': system_prompt}]
    for user_msg, ai_msg in chat_history:
        messages.append({'role': 'user', 'content': user_msg})
        messages.append({'role': 'assistant', 'content': ai_msg})
    messages.append({'role': 'user', 'content': msg})
    return messages

# Function to generate model predictions.
def predict(message, history):
    # Formatting the input for the model.
    messages = build_input_from_chat_history(history, message)
    input_ids = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            return_tensors="ms",
            tokenize=True
        )
    streamer = TextIteratorStreamer(tokenizer, timeout=300, skip_prompt=True, skip_special_tokens=True)
    generate_kwargs = dict(
        input_ids=input_ids,
        streamer=streamer,
        max_new_tokens=1024,
        do_sample=True,
        top_p=0.9,
        temperature=0.1,
        num_beams=1,
    )
    t = Thread(target=model.generate, kwargs=generate_kwargs)
    t.start()  # Starting the generation in a separate thread.
    partial_message = ""
    for new_token in streamer:
        partial_message += new_token
        if '</s>' in partial_message:  # Breaking the loop if the stop token is generated.
            break
        yield partial_message

## 启动聊天机器人

下面我们将启动一个基于Gradio的聊天界面，用于与聊天机器人进行交互。在浏览器中打开链接 [http://127.0.0.1:7860](http://127.0.0.1:7860)，开始与聊天机器人的交互。您可以在页面下方的消息输入框中输入任何问题，或者点击页面下方 **Examples** 中预设的问题，然后点击 **Submit** 按钮与聊天机器人进行对话。注意，第一次回答需要较长时间的加载（约1~2分钟），请耐心等待。

In [ ]:
#install gradio

!pip install gradio==4.44.0

In [3]:
import gradio as gr
# Setting up the Gradio chat interface.
gr.ChatInterface(predict,
                 title="Qwen1.5-0.5b-Chat",
                 description="问几个问题",
                 examples=['你是谁？', '介绍一下华为公司']
                 ).launch()  # Launching the web interface.

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


/home/HwHiAiUser/.local/lib/python3.9/site-packages/gradio/analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.44.0, however version 4.44.1 is available, please upgrade. 
--------
  warnings.warn(


本案例已同步上线 [GitHub 仓](https://github.com/mindspore-courses/orange-pi-mindspore/tree/master/Online/14-qwen1.5-0.5b)，更多案例开发亦可参考该仓库。

本案例运行所需环境：

- **硬件**：香橙派 AIpro 16G 8-12T 开发板
- **镜像**：香橙派官网 Ubuntu 镜像
- **CANN**：8.0.RC3.alpha002
- **MindSpore**：2.4.10